# Simulating voting rules (Python 3.10.10)

## Introduction

Voting rules can be confusing. To illustrate the possible voting rules as discussed in the lectures, we simulate the voting rules and the effects they can have. In this demo, "agent" and "voter" is used interchangably.

In [ ]:
import random
import numpy as np
import pandas as pd
import string
from collections import Counter

## Model definition

### Defining functions

We first define the functions `create_voters` and `create_alternatives` to create specific sets $N$ and $A$, respectively.

In [ ]:
def create_voters(n_voters: int = 0) -> list[int]:
    """
    Create a list of voters of size `n_voters`.
    If `n_voters` is equal to 0, a random number of voters in the range
    [3, 10] will be created.
    """

    assert isinstance(n_voters, int), 'Number of voters must be an integer'
    assert n_voters >= 0, 'Number of voters must be zero or positive'

    if n_voters == 0:
        n_voters = random.randint(3, 10)

    return [i for i in range(n_voters)]


def create_alternatives(n_alternatives: int = 0) -> list[str]:
    """
    Create a list of alternatives of size `n_alternatives`.
    If `n_alternatives` is equal to 0, a random number of alternatives in the range
    [3, 10] will be created.
    """

    assert isinstance(n_alternatives, int), 'Number of alternatives must be an integer'
    assert n_alternatives >= 0, 'Number of alternatives must be zero or positive'
    assert n_alternatives <= 26, 'Number of alternatives must be less than or equal to 26'

    if n_alternatives == 0:
        n_alternatives = random.randint(3, 10)

    return list(string.ascii_lowercase[:n_alternatives])


print(f'{create_voters(5)=}')
print(f'{create_alternatives(5)=}')

create_voters(5)=[0, 1, 2, 3, 4]
create_alternatives(5)=['a', 'b', 'c', 'd', 'e']


We define a function `create_preference_profile` for creating preference profiles. For each agent $i \in N$ it creates a preference $\succ_i$ containing alternatives from the set $A$:

In [ ]:
def create_profile(voters: list[int], alternatives: list[str]) -> dict:
    """
    Create a preference profile for a given number of voters and alternatives.
    Randomized on each call.
    """

    profile = dict()

    for voter in voters:
        profile[voter] = ''.join(random.sample(alternatives, k=len(alternatives)))

    return profile

Functions for getting the top choice of a voter $i$ and the bottom choice: $top(\succ_i) = a_{i_1}$ and $bot(\succ_i) = a_{i_{|A|}}$

In [ ]:
def top_choice(profile: dict, voter: int) -> str:
    """
    Returns the top choice of a voter.
    """

    return profile[voter][0]

def bottom_choice(profile: dict, voter: int) -> str:
    """
    Returns the bottom choice of a voter.
    """

    return profile[voter][-1]

## Description of a sample preference profile

In [ ]:
# sample preference profile for 4 voters and 3 alternatives
n_voters = 4
n_alternatives = 3
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)
print('Sample profile:')
print(f'{profile=}')


print('\nPreferences by voter:')

for voter in voters:
    print(f'Voter {voter}: {profile[voter]}')


# count the number of each preference
preference_count = Counter(profile.values())

print('\nPreferences by frequency:')

for preference, count in preference_count.most_common():
    print(f'{preference}: {count}')

Sample profile:
profile={0: 'bca', 1: 'acb', 2: 'cab', 3: 'abc'}

Preferences by voter:
Voter 0: bca
Voter 1: acb
Voter 2: cab
Voter 3: abc

Preferences by frequency:
bca: 1
acb: 1
cab: 1
abc: 1


## The dictatorship rule $F_{DCT}$

For any profile $R$: $F_{DCT}(R) = top(\succ_i)$, with voter $i \in N$ being the "dictator".

In [ ]:
def dictatorship_rule(profile: dict, dictator: int) -> str:
    """
    This rule returns the top choice of the dictator specified.
    """

    return top_choice(profile, dictator)

In [ ]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

dictator = random.choice(voters)

print('\nDictatorship rule:')
print(f'Dictator: {dictator}')

winner = dictatorship_rule(profile, dictator)
print(f'Winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'abc', 1: 'abc', 2: 'acb', 3: 'cab'}
--------------------------------------------------------------------------------

Dictatorship rule:
Dictator: 0
Winner: a


## The majority rule $F_{MAJ}$

The majority rule states that for any profile $R$: $F_{MAJ}(R) = x$, such that $top(\succ_i) = x$ for a strict majority of voters in $N$. A strict majority is the the half of the voters plus one.

The majority rule does not always have an outcome. In that case, the function `majority_rule` returns `None`. This can happen when `n_voters` is even and/or `n_alternatives` is greater than 2.

In [ ]:
def majority_rule(profile: dict) -> str | None:
    """
    This rule returns the majority top choice.
    If there is no majority, it returns None.
    """

    majority = len(profile) // 2 + 1

    top_choices = [top_choice(profile, voter) for voter in profile]
    top_choice_count = Counter(top_choices)

    for top, count in top_choice_count.items():
        if count >= majority:
            return top

    return None

In [ ]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

print('\nMajority rule:')

winner = majority_rule(profile)
print(f'Winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'abc', 1: 'abc', 2: 'acb', 3: 'cab'}
--------------------------------------------------------------------------------

Majority rule:
Winner: a


## The plurality rule $F_{PLU}$

The plurality rule states that for any profile $R$: $F_{PLU}(R) = \text{argmax}_{x \in A}p_R(x)$, where $p_R(x) = | \{ i \in N | top(\succ_i) = x \} |$. That is, the plurality rule extracts the alternatives that are the most at the top of the voters' preferences.

This implementation of `plurality_rule` returns a random winner in case of a tie. This results in the fact that this rule always has a winner.

In [ ]:
def plurality_rule(profile: dict) -> str:
    """
    This rule returns the plurality top choice.
    If there are ties, return a random winner.
    """

    # construct a list of all the top choices, and count them
    top_choices = [top_choice(profile, voter) for voter in profile]
    choice_votes = Counter(top_choices)

    # find the maximum number of votes
    max_votes = max(choice_votes.values())

    # get winners
    winners = [c for c, v in choice_votes.items() if v == max_votes]

    return random.choice(winners)

In [ ]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

print('\nPlurality rule:')

winner = plurality_rule(profile)
print(f'Winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'abc', 1: 'abc', 2: 'acb', 3: 'cab'}
--------------------------------------------------------------------------------

Plurality rule:
Winner: a


### Duverger's law

The two-party system in the United States is a result of the single-ballot plurality-rule voting system that is in place there. The president is chosen with this voting rule. To illustrate how unfavorable this voting system is, we will simulate a country with 100 citizens and varying between 5, 4, 3 and 2 alternatives.

#### 5 alternatives

In [ ]:
random.seed(27)

n_voters = 100
n_alternatives = 5
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])
winner_on_bottom_2 = sum([1 for voter in profile if winner in profile[voter][-2:]])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')
print(f'In bottom 2 votes: {winner_on_bottom_2} times')



Preferences by frequency:
cadeb:  6   bcdea:  3   cbdae:  3   debac:  3   eadbc:  3   edcab:  3   
acdbe:  2   daceb:  2   dacbe:  2   adbec:  2   dceab:  2   bdcae:  2   
eadcb:  2   cedab:  2   daebc:  2   ebadc:  2   cdabe:  2   bdcea:  2   
bcead:  2   acedb:  2   bdaec:  2   edbac:  2   bcaed:  2   bdeca:  2   
cdeba:  2   baced:  2   bdace:  2   edcba:  2   eacbd:  2   decab:  2   
dcbae:  1   cdaeb:  1   ebcad:  1   cdbea:  1   caedb:  1   cabed:  1   
eabdc:  1   badec:  1   bcdae:  1   edbca:  1   dbace:  1   becda:  1   
dcaeb:  1   eacdb:  1   abdce:  1   cedba:  1   dbcae:  1   aedbc:  1   
aecdb:  1   ecbda:  1   acdeb:  1   bceda:  1   debca:  1   deacb:  1   
cbade:  1   edabc:  1   edacb:  1   adcbe:  1   aebcd:  1   bedac:  1   
daecb:  1   

Plurality rule winner: b
Top vote: 24 times
Bottom vote: 30 times
In bottom 2 votes: 51 times


As we can see, the plurality rule winner for this profile is alternative $b$. However, even though $b$ reached the top of the voter's preference 24 times, they were at the bottom of the preference of 30 voters. This shows that the plurality rule can provide terrible winners that do not align with many people's preferences. Even worse, however, is that in 51 out of 100 voters, the winner $b$ was either one of the bottom two alternatives in their preference.

#### 4 alternatives

In [ ]:
random.seed(706)

n_voters = 100
n_alternatives = 4
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])
winner_on_bottom_2 = sum([1 for voter in profile if winner in profile[voter][-2:]])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')
print(f'In bottom 2 votes: {winner_on_bottom_2} times')



Preferences by frequency:
dabc:  8   cdba:  7   cadb:  7   abcd:  6   badc:  6   dacb:  5   
cabd:  5   dbca:  5   cdab:  5   dcba:  5   adbc:  4   bcda:  4   
bcad:  4   bacd:  4   abdc:  4   bdca:  4   cbad:  4   acbd:  3   
dcab:  3   dbac:  3   cbda:  2   adcb:  1   bdac:  1   

Plurality rule winner: c
Top vote: 30 times
Bottom vote: 26 times
In bottom 2 votes: 51 times


In the above voting simulation, we can again see that the majority of voters have written down the winner ($c$) to be in the bottom half of their preference. This time, the amount of times that alternative $c$ is at the top of preferences is more than the amount of times $c$ is at the bottom of preferences.

## The plurarity with runoff rule $F_{RUN}$

In [ ]:
...

Ellipsis

## The Condorcet rule $F_{CON}$

In [ ]:
def condorcet_matchups(profile: dict) -> np.ndarray:
    alternatives = list(list(profile.values())[0])
    alternatives.sort()

    matchups = np.identity(len(alternatives))

    for preference in profile.values():
        idxs = np.array([preference.index(a) for a in alternatives])

        for i in range(len(alternatives)):
            wins = np.where(idxs[i] < idxs, 1, 0)
            losses = np.where(idxs[i] > idxs, -1, 0)

            matchups[i] += wins + losses

    winners = np.where(matchups > 0, 1, 0)
    return matchups, winners

def condorcet_rule(profile: dict) -> tuple:
    alternatives = list(list(profile.values())[0])
    alternatives.sort()

    matchups, winners = condorcet_matchups(profile)
    winner = np.nonzero(np.all(winners > 0, 1))[0]

    if len(winner) == 0:
        winner = None
    else:
        winner = alternatives[winner]

    return winner, (matchups, winners)

In [ ]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

print('\n Condorcet rule:')

winner, results = condorcet_rule(profile)
print(f'Winner: {winner}')

print('Head-to-head victories: ')
print(results[1])
print(results[0])

n_voters=4
n_alternatives=3
profile={0: 'bca', 1: 'acb', 2: 'cab', 3: 'abc'}
--------------------------------------------------------------------------------

 Condorcet rule:
Winner: None
Head-to-head victories: 
[[1 1 0]
 [0 1 0]
 [0 0 1]]
[[ 1.  2.  0.]
 [-2.  1.  0.]
 [ 0.  0.  1.]]


## The Borda rule $F_{BOR}$

In [ ]:
def borda_score(profile: dict) -> dict:
    br = dict()
    example_preference = list(list(profile.values())[0])
    example_preference.sort()

    n_alternatives = len(example_preference)

    for a in example_preference:
        br[a] = 0

    for preference in profile.values():
        for k, a in enumerate(preference):
            br[a] += n_alternatives - k - 1

    return br


def borda_rule(profile: dict) -> tuple:
    scores = borda_score(profile)
    winner = max(scores, key=scores.get)

    return winner, scores

In [ ]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

print('\n Borda rule:')

winner, scores = borda_rule(profile)
print(f'Winner: {winner}')

print('\n Borda scores: ')
for n, a in scores.items():
    print(f'Alternative {n}: {a}')

n_voters=100
n_alternatives=4
profile={0: 'cdba', 1: 'dabc', 2: 'cadb', 3: 'adbc', 4: 'abcd', 5: 'bcda', 6: 'bcad', 7: 'cdba', 8: 'bacd', 9: 'acbd', 10: 'cdba', 11: 'dacb', 12: 'abdc', 13: 'cabd', 14: 'dcab', 15: 'bacd', 16: 'acbd', 17: 'cabd', 18: 'cdba', 19: 'dacb', 20: 'cabd', 21: 'dbca', 22: 'badc', 23: 'cdba', 24: 'cadb', 25: 'dabc', 26: 'cdab', 27: 'cdab', 28: 'bcad', 29: 'dcba', 30: 'dcab', 31: 'dcba', 32: 'dabc', 33: 'dabc', 34: 'adcb', 35: 'dcba', 36: 'badc', 37: 'dabc', 38: 'abcd', 39: 'abdc', 40: 'dabc', 41: 'dabc', 42: 'abdc', 43: 'bcda', 44: 'adbc', 45: 'bdca', 46: 'abcd', 47: 'badc', 48: 'cadb', 49: 'cbda', 50: 'cbad', 51: 'adbc', 52: 'bcad', 53: 'dabc', 54: 'bdca', 55: 'dacb', 56: 'bcda', 57: 'badc', 58: 'cbad', 59: 'bdca', 60: 'cdab', 61: 'bdac', 62: 'dbca', 63: 'dbac', 64: 'bcda', 65: 'bdca', 66: 'dcba', 67: 'dacb', 68: 'dbca', 69: 'cadb', 70: 'dacb', 71: 'dbca', 72: 'dcba', 73: 'cdab', 74: 'abcd', 75: 'dbac', 76: 'badc', 77: 'badc', 78: 'dcab', 79: 'cdba', 80: 'abcd',

## Various scoring rules

In [ ]:
def score_points(profile: dict, scores: list) -> dict:
    s = dict()
    alt = list(list(profile.values())[0])
    alt.sort()

    n_alt = len(alt)

    for a in alt:
        s[a] = 0

    for pref in profile.values():
        for p, a in zip(scores, pref):
            s[a] += p

    return s

def scoring_rule(profile, scores) -> tuple:
    s = score_points(profile, scores)
    winner = max(s, key=s.get)

    return winner, s

In [ ]:
score = [1, 1, 0, 0]
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')
print('-' * 80)

print('\n Scoring rule:')

winner, scores = scoring_rule(profile, score)
print(f'Winner: {winner}')

print('\n Scores: ')
for n, a in scores.items():
    print(f'Alternative {n}: {a}')

n_voters=100
n_alternatives=4
profile={0: 'cdba', 1: 'dabc', 2: 'cadb', 3: 'adbc', 4: 'abcd', 5: 'bcda', 6: 'bcad', 7: 'cdba', 8: 'bacd', 9: 'acbd', 10: 'cdba', 11: 'dacb', 12: 'abdc', 13: 'cabd', 14: 'dcab', 15: 'bacd', 16: 'acbd', 17: 'cabd', 18: 'cdba', 19: 'dacb', 20: 'cabd', 21: 'dbca', 22: 'badc', 23: 'cdba', 24: 'cadb', 25: 'dabc', 26: 'cdab', 27: 'cdab', 28: 'bcad', 29: 'dcba', 30: 'dcab', 31: 'dcba', 32: 'dabc', 33: 'dabc', 34: 'adcb', 35: 'dcba', 36: 'badc', 37: 'dabc', 38: 'abcd', 39: 'abdc', 40: 'dabc', 41: 'dabc', 42: 'abdc', 43: 'bcda', 44: 'adbc', 45: 'bdca', 46: 'abcd', 47: 'badc', 48: 'cadb', 49: 'cbda', 50: 'cbad', 51: 'adbc', 52: 'bcad', 53: 'dabc', 54: 'bdca', 55: 'dacb', 56: 'bcda', 57: 'badc', 58: 'cbad', 59: 'bdca', 60: 'cdab', 61: 'bdac', 62: 'dbca', 63: 'dbac', 64: 'bcda', 65: 'bdca', 66: 'dcba', 67: 'dacb', 68: 'dbca', 69: 'cadb', 70: 'dacb', 71: 'dbca', 72: 'dcba', 73: 'cdab', 74: 'abcd', 75: 'dbac', 76: 'badc', 77: 'badc', 78: 'dcab', 79: 'cdba', 80: 'abcd',